In [1]:
%pip install --force-reinstall -q -r ./utils/requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.29.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.36.22 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-common 1.2 requires psutil<7.0.0,>=5.7.3, but you have psutil 7.0.0 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-m

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import os
import sys
import time
import boto3
import logging
import requests
import pprint
import json
import time
import warnings
warnings.filterwarnings('ignore')

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

In [4]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '010117700078')

In [5]:
# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-multifunctional-chatbot-kb-{suffix}"
knowledge_base_description = "Multifunctional Chatbot Knowledge Base."

bucket_name = f'{knowledge_base_name}-{account_id}'
intermediate_bucket_name = f'{knowledge_base_name}-intermediate-{account_id}'


In [6]:
data_bucket_name = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.

data_sources=[{"type": "S3", "bucket_name": data_bucket_name}]

In [7]:
import importlib
import utils.knowledge_base
importlib.reload(utils.knowledge_base)
from utils.knowledge_base import BedrockKnowledgeBase

print(suffix)

7220345


In [8]:
# Creating Knowledge Base, may take a few mins.

knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
Creating bucket bedrock-kb-7220345-1
Step 2 - Creating Knowledge Base Execution Role and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
Creating collection...
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Sleeping for a minute to ensure data access rules have been enforced
Step 5 - Creating OSS Vector Index


[2025-02-17 22:09:20,975] p135 {base.py:258} INFO - PUT https://3m5htfwhxx9i1hcl2f3j.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-7220345-f [status:200 request:1.245s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-7220345-f',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 2, 17, 22, 10, 21, 105567, tzinfo=tzlocal()),
  'description': 'Multifunctional Chatbot Knowledge Base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:010117700078:knowledge-base/EHSH1Q38GZ',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'EHSH1Q38GZ',
  'name': 'bedrock-multifunctional-chatbot-kb-7220345',
  'roleArn': 'arn:aws:iam::010117700078:role/BedrockExecutionRoleForKnowledgeBase_7220345-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-west-2:010117700078:collection/3m5htfwhxx9i1hcl2f3j',
                        

In [9]:
def download_and_upload_squad_sample(bucket_name):
    # Download the partial SQuAD dataset
    url = "https://raw.githubusercontent.com/rajpurkar/SQuAD-explorer/master/dataset/dev-v2.0.json"
    response = requests.get(url)
    data = response.json()
    
    # Select 100 as sample
    sample_data = {
        "data": data["data"][:100]
    }
    
    # creating the temp files locally
    with open("/tmp/squad_sample.json", "w") as f:
        json.dump(sample_data, f)
    
    # uplaoding to s3
    s3_client = boto3.client('s3')
    s3_client.upload_file("/tmp/squad_sample.json", bucket_name, "squad_sample.json")

download_and_upload_squad_sample(data_bucket_name)

In [11]:
## Start the ingestion job, embedding the data sources of s3 to opensearch database.
knowledge_base.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'LSDYTJ4PPP',
  'ingestionJobId': '9SUPBWDEI9',
  'knowledgeBaseId': 'EHSH1Q38GZ',
  'startedAt': datetime.datetime(2025, 2, 17, 22, 10, 25, 848380, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 2, 17, 22, 12, 23, 768670, tzinfo=tzlocal())}


In [12]:
## Testing the Knowledge Base:

bedrock_agent = boto3.client('bedrock-agent-runtime')
kb_id = knowledge_base.knowledge_base['knowledgeBaseId']

def simple_kb_test(kb_id, query_text):
    try:
        query = {
            "text": query_text 
        }

        response = bedrock_agent.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery=query,  # 传入查询字典
            retrievalConfiguration={
                "vectorSearchConfiguration": {
                    "numberOfResults": 5,
                } 
            }
        )
        
        print(f"\nQuery: {query_text}")
        print("\nRetrieved results:")
        for i, result in enumerate(response['retrievalResults'], 1):
            print(f"\nResult {i}:")
            print(f"Score: {result['score']}")
            print(f"Content: {result['content']}")
            
    except Exception as e:
        print(f"Error: {e}")

test_query = "What is the major context of the SQuAD dataset?"

simple_kb_test(kb_id, test_query)


Query: What is the major context of the SQuAD dataset?

Retrieved results:

Result 1:
Score: 0.37877417
Content: {'text': ', "id": "57284b904b864d19001648e2", "answers": [{"text": "the Main Quadrangles", "answer_start": 92}, {"text": "Main Quadrangles", "answer_start": 96}, {"text": "the Main Quadrangles", "answer_start": 92}, {"text": "the Main Quadrangles", "answer_start": 92}], "is_impossible": false}, {"question": "How many quadrangles does the Main Quadrangles have?", "id": "57284b904b864d19001648e3", "answers": [{"text": "six", "answer_start": 273}, {"text": "six quadrangles", "answer_start": 273}, {"text": "six", "answer_start": 273}, {"text": "six", "answer_start": 273}], "is_impossible": false}, {"question": "Who helped designed the Main Quadrangles?"', 'type': 'TEXT'}

Result 2:
Score: 0.3762661
Content: {'text': '": "explore computer networking", "answer_start": 190}], "is_impossible": false}, {"question": "What completed the triad ", "id": "5726414e271a42140099d7e6", "answ

In [13]:
## Creating Inference Profile for Amazon Nova Pro model.

nova_pro_profile_name = f'bedrock-kb-nova-pro-profile-{suffix}' 
profile_arn = knowledge_base.create_nova_inference_profile(nova_pro_profile_name, throughput=1)

Current status: ACTIVE
Inference profile created successfully: arn:aws:bedrock:us-west-2:010117700078:application-inference-profile/732bbsotu6s5


In [14]:
## Creating Deepseek model with Sagemaker JumpStart

from sagemaker.jumpstart.model import JumpStartModel

model_id = "deepseek-llm-r1-distill-qwen-1-5b"
my_model = JumpStartModel(model_id=model_id, instance_type='ml.g5.2xlarge')

predictor = my_model.deploy()

deepseek_sagemaker_endpoint = predictor.endpoint_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Using model 'deepseek-llm-r1-distill-qwen-1-5b' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
[2025-02-17 22:13:08,069] p135 {cache.py:625} WARNING - Using model 'deepseek-llm-r1-distill-qwen-1-5b' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
[2025-02-17 22:13:08,118] p135 {session.py:4094} INFO - Creating model with name: deepseek-llm-r1-distill-qwen-1-5b-2025-02-17-22-13-08-116
[2025-02-17 22:13:08,911] p135 {session.py:5889} INFO - Creating endpoint-config with name deepseek-llm-r1-distill-qwen-1-5b-2025-02-17-22-13-08-117
[2025-02-17 22:13:09,247] p135 {session.py:4711} INFO - Creating endpoint with name deepseek-llm-r1-distill-qwen-1-5b-2025-02-17-22-13-08-117


----------!

In [16]:
## Writing configurations into a config file, then access by the  Streamlit APP

config = {
    "kb_id": kb_id,
    "nova_pro_profile_arn": profile_arn,
    "nova_pro_model_id": "amazon.nova-pro-v1:0",
    "sagemaker_endpoint": deepseek_sagemaker_endpoint,
    "sagemaker_ep_arn" : f"arn:aws:sagemaker:{region}:{account_id}:endpoint/{deepseek_sagemaker_endpoint}",
    "region_name": region
}

with open('utils/tmp_config.json', 'w') as f:
    json.dump(config, f, indent=4)




In [ ]:
print("=============================== All resources have been completed, please starting your demo ==============================\n")

#### Start a Terminal Session on Jupyterlab, then execute the below command:

```bash
pip install streamlit

streamlit run demo-dev/app.py 
```

#### To access the Streamlit Web Application via:

1. Copy & paste the URL of the Sagemaker Studio Jupyterlab web URL, eg:

https://xxxxxxxxxxxxx.studio.us-west-2.sagemaker.aws/jupyterlab/default/lab/.../lab-code.ipynb


2. Update the url as below format, and access the url via a new browser tab:

https://xxxxxxxxxxxxx.studio.us-west-2.sagemaker.aws/jupyterlab/default/proxy/8501/


In [26]:
## Clean up


# print("===============================Starting Clean up==============================\n")
# predictor.delete_predictor()
# knowledge_base.delete_kb(delete_s3_bucket=True)

# file_path = 'utils/tmp_config.json'
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File {file_path} has been deleted successfully.")
# else:
#     print(f"File {file_path} does not exist.")

===============================Knowledge base with fixed chunking==============================

File utils/tmp_config.json has been deleted successfully.
